### <데이터 수집 URL>

- 데이터 수집 기관 : 국가교통 데이터 오픈 마켓

- URL : https://www.bigdata-transportation.kr

- 검색어 : 포항시 BIS 교통카드 사용내역

- 회원가입 필요함

### <데이터셋 1개만 샘플링 분석하기>

- 다운로드한 데이터가 많은 경우에는 한 개의 파일을 샘플링 후 -> 향후 반복문을 이용해서 통합하면 된다.

- trfcard (0) 폴더 내의 데이터를 읽어들여서 샘플이 분석 진행

In [1]:
### trfcard (0) 폴더 내에 csv파일 읽어들이기
# - 변수명 : df_bus_card_org
import pandas as pd

file_path = './01_data_org/trfcard (0)/trfcard.csv'
df_bus_card_org = pd.read_csv(file_path, encoding='utf-8')
df_bus_card_org.head(1)

,on_date,off_date,route_name,descr,age_type,trans_yn,addfee_yn,start_bstop,start_gps_x,start_gps_y,end_bstop,end_gps_x,end_gps_y
0,20200102051049,20200102051844,131,양덕-송도-시청-양학-양덕,일반,N,N,양덕차고지,129.401693,36.077258,동부초등학교,129.380866,36.062209


In [2]:
### 데이터 기본정보 및 기초통계 확인하기
# - 결측치 및 이상데이터 확인 가능
df_bus_card_org.info()
df_bus_card_org.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16185 entries, 0 to 16184
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   on_date      16185 non-null  int64  
 1   off_date     16185 non-null  int64  
 2   route_name   16185 non-null  object 
 3   descr        16185 non-null  object 
 4   age_type     16164 non-null  object 
 5   trans_yn     16185 non-null  object 
 6   addfee_yn    16185 non-null  object 
 7   start_bstop  16185 non-null  object 
 8   start_gps_x  16185 non-null  float64
 9   start_gps_y  16185 non-null  float64
 10  end_bstop    16185 non-null  object 
 11  end_gps_x    16185 non-null  float64
 12  end_gps_y    16185 non-null  float64
dtypes: float64(4), int64(2), object(7)
memory usage: 1.6+ MB


,on_date,off_date,start_gps_x,start_gps_y,end_gps_x,end_gps_y
count,1.618500e+04,1.618500e+04,16185.000000,16185.000000,16185.000000,16185.000000
mean,2.020010e+13,2.020010e+13,129.369118,36.034985,129.368939,36.031927
std,4.127644e+04,4.175148e+04,0.048054,0.043280,0.040777,0.033987
min,2.020010e+13,2.020010e+13,129.052958,35.804119,129.052958,35.804119
25%,2.020010e+13,2.020010e+13,129.350290,36.012537,129.353856,36.013250
50%,2.020010e+13,2.020010e+13,129.365461,36.031720,129.365281,36.034110
75%,2.020010e+13,2.020010e+13,129.383462,36.064088,129.370823,36.046050
max,2.020010e+13,2.020010e+13,129.578137,36.254158,129.578137,36.244762


- 'age_type' 특성에 결측치가 존재하는 것으로 보이며 이외 이상치는 확인은 어려움

- 실제 분석 시에는 결측치, 이상치, 중복 데이터에 대한 프로그램적 확인을 실시해야 한다.

### <특성 이름을 한글명으로 반환하기 -> 변경 순서>

1. 제공된 메타정의서 파일 읽어들이기

2. 영문과 한글명이 있는 특성 부분만 읽어들이면 된다.

3. 딕셔너리로 변환하기 : 메타정의서의 영문명를 key로, 한글명을 value로 생성 

4. 데이터프레임의 `rename()`함수를 이용하여 컬럼명 변환하기

In [3]:
### 제공된 메타정의서 파일 읽어들이기
### 영문과 한글명이 있는 특성 부분만 읽어들이기
# - 데이터프레임 변수명 : df_bus_card_col_org

df_bus_card_col_org = pd.read_excel('./01_data_org/trfcard (0)/trfcard_columns.xlsx',header=2,usecols='B:C')
df_bus_card_col_org

,컬럼명 (영문),컬럼명 (한글)
0,on_date,승차시각
1,off_date,하차시각
2,route_name,노선명
3,descr,노선설명
4,age_type,승객연령
5,trans_yn,환승여부
6,addfee_yn,추가운임여부
7,start_bstop,승차정류장
8,start_gps_x,승차정류장 GPS X
9,start_gps_y,승차정류장 GPS Y


In [4]:
### 딕셔너리로 변환하기 : 메타정의서의 영문명를 key로, 한글명을 value로 생성
# - 딕셔너리 변수명 : df_bus_card_col_dict
# - 예시 : {'on_date': '승차시각', 'off_date': '하차시각',...}

df_bus_card_col_dict ={}
for i in range(len(df_bus_card_col_org)):
    df_bus_card_col_dict[df_bus_card_col_org['컬럼명 (영문)'][i]] = df_bus_card_col_org['컬럼명 (한글)'][i]
df_bus_card_col_dict

{'on_date': '승차시각',
 'off_date': '하차시각',
 'route_name': '노선명',
 'descr': '노선설명',
 'age_type': '승객연령',
 'trans_yn': '환승여부',
 'addfee_yn': '추가운임여부',
 'start_bstop': '승차정류장',
 'start_gps_x': '승차정류장 GPS X',
 'start_gps_y': '승차정류장 GPS Y',
 'end_bstop': '하차정류장',
 'end_gps_x': '하차정류장 GPS X',
 'end_gps_y': '하차정류장 GPS Y'}

In [5]:
### 강사님 코드 (1)
df_bus_card_col_dict ={}

for i in range(0,len(df_bus_card_col_org),1):
    df_bus_card_col_dict[df_bus_card_col_org.iloc[i,0]] = df_bus_card_col_org.iloc[i,1]
df_bus_card_col_dict

{'on_date': '승차시각',
 'off_date': '하차시각',
 'route_name': '노선명',
 'descr': '노선설명',
 'age_type': '승객연령',
 'trans_yn': '환승여부',
 'addfee_yn': '추가운임여부',
 'start_bstop': '승차정류장',
 'start_gps_x': '승차정류장 GPS X',
 'start_gps_y': '승차정류장 GPS Y',
 'end_bstop': '하차정류장',
 'end_gps_x': '하차정류장 GPS X',
 'end_gps_y': '하차정류장 GPS Y'}

In [6]:
### 강사님 코드 (2)
df_bus_card_col_dict = dict(df_bus_card_col_org.to_numpy())
df_bus_card_col_dict

{'on_date': '승차시각',
 'off_date': '하차시각',
 'route_name': '노선명',
 'descr': '노선설명',
 'age_type': '승객연령',
 'trans_yn': '환승여부',
 'addfee_yn': '추가운임여부',
 'start_bstop': '승차정류장',
 'start_gps_x': '승차정류장 GPS X',
 'start_gps_y': '승차정류장 GPS Y',
 'end_bstop': '하차정류장',
 'end_gps_x': '하차정류장 GPS X',
 'end_gps_y': '하차정류장 GPS Y'}

In [7]:
### 데이터프레임의 rename()함수를 이용하여 컬럼명 변환하기
df_bus_card_kor=df_bus_card_org.rename(columns=df_bus_card_col_dict)
df_bus_card_kor.head(1)

,승차시각,하차시각,노선명,노선설명,승객연령,환승여부,추가운임여부,승차정류장,승차정류장 GPS X,승차정류장 GPS Y,하차정류장,하차정류장 GPS X,하차정류장 GPS Y
0,20200102051049,20200102051844,131,양덕-송도-시청-양학-양덕,일반,N,N,양덕차고지,129.401693,36.077258,동부초등학교,129.380866,36.062209


### <현황 분석을 위한 소주제 정하기 : 특성 확인>

[버스 이용량 현황 분석]

- 기준월 및 기준일자별 버스 이용량 분석 비교

- 기준일 및 기준시간별 버스 이용량 분석 비교

- 기준시간 및 분별 버스 이용량 분석 비교

[버스 내 체류시간 현황 분석]

[승/하차 정류장별 버스 내 체류시간 현황 분석]

In [8]:
df_bus_card_kor.head(1)

,승차시각,하차시각,노선명,노선설명,승객연령,환승여부,추가운임여부,승차정류장,승차정류장 GPS X,승차정류장 GPS Y,하차정류장,하차정류장 GPS X,하차정류장 GPS Y
0,20200102051049,20200102051844,131,양덕-송도-시청-양학-양덕,일반,N,N,양덕차고지,129.401693,36.077258,동부초등학교,129.380866,36.062209


In [9]:
df_bus_card_kor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16185 entries, 0 to 16184
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   승차시각         16185 non-null  int64  
 1   하차시각         16185 non-null  int64  
 2   노선명          16185 non-null  object 
 3   노선설명         16185 non-null  object 
 4   승객연령         16164 non-null  object 
 5   환승여부         16185 non-null  object 
 6   추가운임여부       16185 non-null  object 
 7   승차정류장        16185 non-null  object 
 8   승차정류장 GPS X  16185 non-null  float64
 9   승차정류장 GPS Y  16185 non-null  float64
 10  하차정류장        16185 non-null  object 
 11  하차정류장 GPS X  16185 non-null  float64
 12  하차정류장 GPS Y  16185 non-null  float64
dtypes: float64(4), int64(2), object(7)
memory usage: 1.6+ MB


- 승차시각 및 하차시각에 대한 특성을 통해서 필요한 월/일/시/분을 추출하기 위해서는 날짜 타입으로 변환하는 것이 좋다.

- 최초 데이터 특성의 타입이 정수형 타입인 경우에는 아래 순서로 변환한다.

    1. 문자열로 먼저 데이터 타입 변환하기
 
    2. 그 다음 날짜 타입으로 변환하기

In [10]:
### 승차시각과 하차시각의 데이터 타입을 -> 문자열로 데이터 타입 변환하기

df_bus_card_kor=df_bus_card_kor.astype({'승차시각' : 'str','하차시각' : 'str'})
df_bus_card_kor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16185 entries, 0 to 16184
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   승차시각         16185 non-null  object 
 1   하차시각         16185 non-null  object 
 2   노선명          16185 non-null  object 
 3   노선설명         16185 non-null  object 
 4   승객연령         16164 non-null  object 
 5   환승여부         16185 non-null  object 
 6   추가운임여부       16185 non-null  object 
 7   승차정류장        16185 non-null  object 
 8   승차정류장 GPS X  16185 non-null  float64
 9   승차정류장 GPS Y  16185 non-null  float64
 10  하차정류장        16185 non-null  object 
 11  하차정류장 GPS X  16185 non-null  float64
 12  하차정류장 GPS Y  16185 non-null  float64
dtypes: float64(4), object(9)
memory usage: 1.6+ MB


In [11]:
### 승차시각과 하차시각의 데이터 타입을 -> 날짜 데이터 타입 변환하기
df_bus_card_kor['승차시각']=pd.to_datetime(df_bus_card_kor['승차시각'])
df_bus_card_kor['하차시각']=pd.to_datetime(df_bus_card_kor['하차시각'])
df_bus_card_kor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16185 entries, 0 to 16184
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   승차시각         16185 non-null  datetime64[ns]
 1   하차시각         16185 non-null  datetime64[ns]
 2   노선명          16185 non-null  object        
 3   노선설명         16185 non-null  object        
 4   승객연령         16164 non-null  object        
 5   환승여부         16185 non-null  object        
 6   추가운임여부       16185 non-null  object        
 7   승차정류장        16185 non-null  object        
 8   승차정류장 GPS X  16185 non-null  float64       
 9   승차정류장 GPS Y  16185 non-null  float64       
 10  하차정류장        16185 non-null  object        
 11  하차정류장 GPS X  16185 non-null  float64       
 12  하차정류장 GPS Y  16185 non-null  float64       
dtypes: datetime64[ns](2), float64(4), object(7)
memory usage: 1.6+ MB


### <현황 분석에 필요한 데이터만 정제하기>

- 사용할 특성 : 승차시각, 하차시각, 승객연령, 승차정류장, 하차정류장

- 새로운 데이터프레임 변수 명 : df_bus_card

In [14]:
df_bus_card = df_bus_card_kor[['승차시각', '하차시각','승객연령','승차정류장','하차정류장']].copy()
df_bus_card.head(1)

,승차시각,하차시각,승객연령,승차정류장,하차정류장
0,2020-01-02 05:10:49,2020-01-02 05:18:44,일반,양덕차고지,동부초등학교


In [15]:
df_bus_card.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16185 entries, 0 to 16184
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   승차시각    16185 non-null  datetime64[ns]
 1   하차시각    16185 non-null  datetime64[ns]
 2   승객연령    16164 non-null  object        
 3   승차정류장   16185 non-null  object        
 4   하차정류장   16185 non-null  object        
dtypes: datetime64[ns](2), object(3)
memory usage: 632.4+ KB


### 현황 분석에 필요한 특성 생성(추가)하기

- 추가할 특성명 : 버스내체류시간(분), 기준년도, 기준월, 기준일, 기준시간, 기준시간(분)

In [26]:
### 버스 내 체류시간(분) 특성 추가하기
#- 버스 내 체류시간(분) = 하차시각 - 승차시각

df_bus_card['버스 내 체류시간(분)'] = round((df_bus_card['하차시각'] - df_bus_card['승차시각']).dt.total_seconds() / 60, 2)
df_bus_card.head(1)

,승차시각,하차시각,승객연령,승차정류장,하차정류장,버스 내 체류시간(분)
0,2020-01-02 05:10:49,2020-01-02 05:18:44,일반,양덕차고지,동부초등학교,7.92


In [29]:
### 기준년도, 기준월, 기준일, 기준시간, 기준시간(분)
df_bus_card['기준년도'] = df_bus_card['승차시각'].dt.year
df_bus_card['기준월'] = df_bus_card['승차시각'].dt.month
df_bus_card['기준일'] = df_bus_card['승차시각'].dt.day
df_bus_card['기준시간'] = df_bus_card['승차시각'].dt.hour
df_bus_card['기준시간(분)'] = df_bus_card['승차시각'].dt.minute
df_bus_card.head(1)

,승차시각,하차시각,승객연령,승차정류장,하차정류장,버스 내 체류시간(분),기준년도,기준월,기준일,기준시간,기준시간(분)
0,2020-01-02 05:10:49,2020-01-02 05:18:44,일반,양덕차고지,동부초등학교,7.92,2020,1,2,5,10


### <다운로드 받은 모든 폴더의 데이터셋을 하나의 데이터프레임으로 통합하기>

- 위에서 샘플로 처리한 순서대로 처리 후에 최종 통합한다.

- trfcard (0) ~ trfcard(79) 전체를 반복하면서 통합한다.

- 최종 통합할 데이터프레임 변수명 : df_bus_card_tot

In [41]:
### 날짜 라이브러리
from datetime import datetime

In [43]:
### 다운로드 받은 모든 폴더의 데이터셋을 하나의 데이터프레임으로 통합하기

### 반복시간 확인하기 : 시작하기
start_date = datetime.today().strftime('%Y-%m-%d %H:%M:%S')

df_bus_card_tot = pd.DataFrame()

for i in range(80):
    file_path = f'./01_data_org/trfcard ({i})/trfcard.csv'
    df_temp = pd.read_csv(file_path,encoding='utf-8')
    df_temp=df_temp.rename(columns=df_bus_card_col_dict)
    df_temp1 = df_temp[['승차시각', '하차시각','승객연령','승차정류장','하차정류장']].copy()
    df_temp1=df_temp1.astype({'승차시각' : 'str','하차시각' : 'str'})
    df_temp1['승차시각']=pd.to_datetime(df_temp1['승차시각'])
    df_temp1['하차시각']=pd.to_datetime(df_temp1['하차시각'])
    df_temp1['버스 내 체류시간(분)'] = round((df_temp1['하차시각'] - df_temp1['승차시각']).dt.total_seconds() / 60, 2)
    df_temp1['기준년도'] = df_temp1['승차시각'].dt.year
    df_temp1['기준월'] = df_temp1['승차시각'].dt.month
    df_temp1['기준일'] = df_temp1['승차시각'].dt.day
    df_temp1['기준시간'] = df_temp1['승차시각'].dt.hour
    df_temp1['기준시간(분)'] = df_temp1['승차시각'].dt.minute
    df_bus_card_tot  = pd.concat([df_bus_card_tot ,df_temp1],ignore_index=True)

end_date = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
print(f'전체 실행 시간 : {end_date} - {start_date}')

df_bus_card_tot

전체 실행 시간 : 2024-03-29 13:47:32 - 2024-03-29 13:47:20


,승차시각,하차시각,승객연령,승차정류장,하차정류장,버스 내 체류시간(분),기준년도,기준월,기준일,기준시간,기준시간(분)
0,2020-01-02 05:10:49,2020-01-02 05:18:44,일반,양덕차고지,동부초등학교,7.92,2020,1,2,5,10
1,2020-01-02 05:13:38,2020-01-02 05:45:49,일반,문덕사거리,새마을금고해도지점,32.18,2020,1,2,5,13
2,2020-01-02 05:15:49,2020-01-02 05:19:30,일반,문덕사거리,성우오토모티브,3.68,2020,1,2,5,15
3,2020-01-02 05:16:17,2020-01-02 05:50:46,일반,부영사랑3차,오거리,34.48,2020,1,2,5,16
4,2020-01-02 05:18:41,2020-01-02 05:23:10,일반,문덕온천,용덕사거리,4.48,2020,1,2,5,18
...,...,...,...,...,...,...,...,...,...,...,...
842603,2020-03-31 23:15:29,2020-03-31 23:20:33,일반,법원사거리,농협하나로클럽,5.07,2020,3,31,23,15
842604,2020-03-31 23:16:51,2020-03-31 23:20:54,일반,LG빌라입구,대잠중앙하이츠,4.05,2020,3,31,23,16
842605,2020-03-31 23:38:06,2020-03-31 23:45:46,일반,대동우방아파트,상대시장,7.67,2020,3,31,23,38
842606,2020-03-31 23:48:33,2020-03-31 23:59:54,일반,해병대서문,부영사랑3차,11.35,2020,3,31,23,48


In [39]:
len(df_bus_card_tot)

842608

In [40]:
df_bus_card_tot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842608 entries, 0 to 842607
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   승차시각          842608 non-null  datetime64[ns]
 1   하차시각          842608 non-null  datetime64[ns]
 2   승객연령          841683 non-null  object        
 3   승차정류장         842608 non-null  object        
 4   하차정류장         842608 non-null  object        
 5   버스 내 체류시간(분)  842608 non-null  float64       
 6   기준년도          842608 non-null  int32         
 7   기준월           842608 non-null  int32         
 8   기준일           842608 non-null  int32         
 9   기준시간          842608 non-null  int32         
 10  기준시간(분)       842608 non-null  int32         
dtypes: datetime64[ns](2), float64(1), int32(5), object(3)
memory usage: 54.6+ MB


### 통합된 파일은 항상 저장해서 사용한다.

- 저장 파일명 : df_bus_card_tot

- 저장 위치 : 02_data_all

In [45]:
df_bus_card_tot.to_csv('./02_data_all/df_bus_card_tot.csv',index=False)

In [46]:
### 저장 잘 됐는지 확인
pd.read_csv('./02_data_all/df_bus_card_tot.csv')

,승차시각,하차시각,승객연령,승차정류장,하차정류장,버스 내 체류시간(분),기준년도,기준월,기준일,기준시간,기준시간(분)
0,2020-01-02 05:10:49,2020-01-02 05:18:44,일반,양덕차고지,동부초등학교,7.92,2020,1,2,5,10
1,2020-01-02 05:13:38,2020-01-02 05:45:49,일반,문덕사거리,새마을금고해도지점,32.18,2020,1,2,5,13
2,2020-01-02 05:15:49,2020-01-02 05:19:30,일반,문덕사거리,성우오토모티브,3.68,2020,1,2,5,15
3,2020-01-02 05:16:17,2020-01-02 05:50:46,일반,부영사랑3차,오거리,34.48,2020,1,2,5,16
4,2020-01-02 05:18:41,2020-01-02 05:23:10,일반,문덕온천,용덕사거리,4.48,2020,1,2,5,18
...,...,...,...,...,...,...,...,...,...,...,...
842603,2020-03-31 23:15:29,2020-03-31 23:20:33,일반,법원사거리,농협하나로클럽,5.07,2020,3,31,23,15
842604,2020-03-31 23:16:51,2020-03-31 23:20:54,일반,LG빌라입구,대잠중앙하이츠,4.05,2020,3,31,23,16
842605,2020-03-31 23:38:06,2020-03-31 23:45:46,일반,대동우방아파트,상대시장,7.67,2020,3,31,23,38
842606,2020-03-31 23:48:33,2020-03-31 23:59:54,일반,해병대서문,부영사랑3차,11.35,2020,3,31,23,48
